### API를 이용한 지하철 탑승현황 분석 기본

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm


In [29]:
def url_print(tdate, uRow=5):
    uKey="7672746155736a6436336451676175"
    url_gibon="http://openapi.seoul.go.kr:8088/"+uKey
    url_obj="/xml/CardSubwayStatsNew/"
    url_row="1/"+str(uRow)+'/'
    url_date=str(tdate)
    url=url_gibon+url_obj+url_row+url_date
    return url



def html_parser(url):
    # url을 이용한 사이트 파씽
    url_xml=requests.get(url)

    if url_xml.status_code != 200:
        print('데이터를 가져오지 못했습니다.')
        efit()

    seoul_sw_soup=bs(url_xml.content, 'html.parser')
    return seoul_sw_soup

def seoul_sw_pandas(seoul_sw_soup):
    xml_row=seoul_sw_soup.find_all('row')
    xml_txt=[]
    for row in xml_row:
        dt=row.find('use_dt').text
        line=row.find('line_num').text
        sub_sta=row.find('sub_sta_nm').text
        ride=row.find('ride_pasgr_num').text
        alight=row.find('alight_pasgr_num').text
        xml_txt.append([dt, line, sub_sta, ride, alight])

    df=pd.DataFrame(xml_txt, columns=['사용일', '라인', '역명', '승차', '하차'])
    return df

    
    
    
    
    
    
    
    
sDt=input('조회 시작일 입력(예:20210708: ')
eDt=input('조회 종료일 입력(예:20210708: ')
url=url_print(sDt, 5)
seoul_sw_soup=html_parser(url)    # 한페이지에 5개의 데이터가 출력된 url 정보 가져오기
#print(url)
#print("========================================================================================")
uRow=seoul_sw_soup.find('list_total_count').text   # 조회된 전체 데이터 개수 추출하기
url=url_print(sDt, uRow)          # 전체 데이터가 출력되는 url 생성
seoul_sw_soup=html_parser(url)    # 한페이지에 추출한 전체 데이터 출력 url 정보 가져오기
#print(url)
df=seoul_sw_pandas(seoul_sw_soup)
print(df)




조회 시작일 입력(예:20210708: 20200108
          사용일   라인    역명     승차     하차
0    20200108  1호선   서울역  58754  53792
1    20200108  1호선    시청  27996  28663
2    20200108  1호선    종각  49926  48608
3    20200108  1호선  종로3가  37405  35497
4    20200108  1호선  종로5가  29338  29116
..        ...  ...   ...    ...    ...
587  20200108  중앙선    오빈    325    302
588  20200108  중앙선    양평   3562   3679
589  20200108  중앙선    원덕    391    390
590  20200108  중앙선    용문   1719   1721
591  20200108  중앙선    지평     55     33

[592 rows x 5 columns]
